In [1]:
import os
from itertools import product
import numpy as np
from Gaugi import load as gaugi_load
from Gaugi import save as gaugi_save

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [2]:
old_ets = np.arange(5)
old_etas = np.arange(5)
#old_ets = np.array([4])
#old_etas = np.array([0])
et_key = 'L2Calo_et'
eta_key = 'L2Calo_eta'
dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97'
homepath = os.path.expanduser('~')
#homepath = 'E:\\data'
datapath = os.path.join(homepath, 'data', dataset)
filepath = os.path.join(datapath, dataset + '_et{et}_eta{eta}.npz')

In [3]:
new_etbins = np.array([15, 20, 30, 40, 50, 100, 1e6])   #Gev
new_etabins = np.array([0., 0.8, 1.37, 1.54, 2.37, 2.5])
new_ets = np.arange(len(new_etbins)+1)
new_dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins'
new_datapath = os.path.join(homepath, 'data', new_dataset)
new_filepath = os.path.join(new_datapath, new_dataset + '_et{et}_eta{eta}.npz')
if not os.path.exists(new_datapath):
    os.makedirs(new_datapath)

In [4]:
refpath = os.path.join(datapath, 'references')
ref_filepath = os.path.join(refpath, dataset + '_et{et}_eta{eta}.ref.pic.gz')
new_refpath = os.path.join(new_datapath, 'references')
new_ref_filepath = os.path.join(new_refpath, new_dataset + '_et{et}_eta{eta}.ref.pic.gz')
if not os.path.exists(new_refpath):
    os.makedirs(new_refpath)

In [5]:
for old_et, old_eta in product(old_ets, old_etas):
    print(f'Reading et {old_et} eta {old_eta}')
    data = dict(np.load(filepath.format(et=old_et, eta=old_eta)))
    et_data_idx = np.where(data['features'] == et_key)[0][0]
    eta_data_idx = np.where(data['features'] == eta_key)[0][0]
    
    if old_et == 4:
        for new_et, new_eta in product(np.array([4,5]), np.array([old_eta])):
            if new_et == 4:
                limit = new_etbins[new_et+1] * 1000     #Mev
                selector = data['data'][:, et_data_idx] < limit
            else:
                limit = new_etbins[new_et] * 1000       #Mev
                selector = data['data'][:, et_data_idx] >= limit
            new_data = {
                'features': data['features'],
                'etBins': new_etbins,
                'etaBins': new_etabins,
                'etBinIdx': new_et,
                'etaBinIdx': new_eta,
                'data': data['data'][selector].copy(),
                'target': data['target'][selector].copy()
            }
            print(f'Dumping et {new_et} eta {new_eta} with {limit} Mev')
            print(f'Dumping data at {new_filepath.format(et=new_et, eta=new_eta)}')
            np.savez(new_filepath.format(et=new_et, eta=new_eta), **new_data)
            print('Dumped data')
            
            ref = gaugi_load(ref_filepath.format(et=old_et, eta=old_eta)).copy()
            ref['etBinIdx'] = new_et
            ref['etBins'] = new_etbins
            ref['etBinIdx'] = new_eta
            ref['etBins'] = new_etabins
            print(f'Dumping ref at {new_ref_filepath.format(et=new_et, eta=new_eta)}')
            ref = gaugi_save(ref, new_ref_filepath.format(et=new_et, eta=new_eta), protocol='savez')
            print('Dumped references')
    else:
        print(f'Dumping et {old_et} eta {old_eta}')
        print(f'Dumping data at {new_filepath.format(et=old_et, eta=old_eta)}')
        data['etBins'] = new_etbins
        np.savez(new_filepath.format(et=old_et, eta=old_eta, **data))
        print('Dumped data')

        ref = gaugi_load(ref_filepath.format(et=old_et, eta=old_eta)).copy()
        ref['etBinIdx'] = old_et
        ref['etBins'] = new_etbins
        ref['etBinIdx'] = old_eta
        ref['etBins'] = new_etabins
        print(f'Dumping ref at {new_ref_filepath.format(et=old_et, eta=old_eta)}')
        ref = gaugi_save(ref, new_ref_filepath.format(et=old_et, eta=old_eta), protocol='savez')
        print('Dumped references')



Reading et 0 eta 0
Dumping et 0 eta 0
Dumping data at /home/lbarranunes/data/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins_et0_eta0.npz
Dumped data
Dumping ref at /home/lbarranunes/data/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins/references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins_et0_eta0.ref.pic.gz
Dumped references
Reading et 0 eta 1
Dumping et 0 eta 1
Dumping data at /home/lbarranunes/data/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins_et0_eta1.npz
Dumped data
Dumping ref at /home/lbarranunes/data/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30bins/references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97.30b

In [6]:
data = dict(np.load(new_filepath.format(et=4, eta=0)))

In [7]:
data.keys()

dict_keys(['features', 'etBins', 'etaBins', 'etBinIdx', 'etaBinIdx', 'data', 'target'])